In [1]:
# copy the lib file in the working directory and load it
from shutil import copyfile
copyfile('target/release/libising.so', 'ising.so')
import ising

In [2]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import threading
import time

In [4]:
L = 24

k_ijl = [(i,j,l) for j in range(2*L) for i in range(2*L) for l in range(3) if (i+j > L-2) and (i+j < 3*L-1)]
ijl_k = {}
for k, ijl in enumerate(k_ijl):
    ijl_k[ijl] = k

In [5]:
A = [[(0,0), (0,0), (-1,0), (0,-1)], 
     [(1,-1), (1,0), (0,0), (0,0)], 
     [(0,1), (-1,1), (0,0), (0,0)]]
B = [[1, 2, 1, 2], [2, 0, 2, 0], [0, 1, 0, 1]]

nn1 = [[(i + A[l][u][0], j + A[l][u][1], B[l][u]) for u in range(4)] for (i,j,l) in k_ijl]

A = [[(-1, 1), (-1, 0), (0, -1), (1, -1)],
     [(0, -1), (1, -1), (1, 0), (0, 1)],
     [(1, 0), (0, 1), (-1, 1), (-1, 0)]]
B = [[1, 2, 1, 2], [2, 0, 2, 0], [0, 1, 0, 1]]

nn2 = [[(i + A[l][u][0], j + A[l][u][1], B[l][u]) for u in range(4)] for (i,j,l) in k_ijl]

# fixing periodic border conditions 
def foo(i, j, l):
    if i == 2*L:
        i = 0
        j += L
    if j == 2*L:
        i += L
        j = 0
    if i+j == L-2:
        i += L
        j += L
    if i+j == 3*L-1:
        i -= L
        j -= L
    if j == -1:
        i -= L
        j = 2*L-1
    if i == -1:
        i = 2*L-1
        j -= L
    return (i, j, l)

nn1 = [[ijl_k[foo(i,j,l)] for (i,j,l) in n] for n in nn1]
nn2 = [[ijl_k[foo(i,j,l)] for (i,j,l) in n] for n in nn2]

In [25]:
H = ising.Hamiltonian([(-1, nn1), (0.5, nn2)])

x = np.random.randint(0, 2, size=(9 * L * L,), dtype=np.int32) * 2 - 1

for beta in np.linspace(0.1, 10, 10):
    for _ in range(100):
        H.sweep(x, 1 / beta)
    print(H.energy(x) / (9 * L * L))

-0.2341820987654321
-1.556712962962963
-1.6134259259259258
-1.6342592592592593
-1.654706790123457
-1.6666666666666667
-1.6666666666666667
-1.6666666666666667
-1.6666666666666667
-1.6666666666666667
